#### Sklearn的安装
- pip install -U scikit-learn
- 前提是要安装好numpy、scipy

安装很顺利，但是导入包时报错如下
> 错误提示：numpy、scipy版本不能太旧，有可能和最新的Sklearn包不兼容，导致报错
> 
>如import sklearn报错
>
>RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb
>
如from sklearn.databasets import * 报错
>
ImportError: cannot import name murmurhash3_32

解决方法：我更新了numpy版本，以上问题解决了
>numpy version：1.15.0
>
scipy Version: 1.1.0
>
scikit-learn Version: 0.19.2

#### Sklearn 包含了很多种机器学习的方式:

- Classification 分类
- Regression 回归
- Clustering 非监督分类
- Dimensionality reduction 数据降维
- Model Selection 模型选择
- Preprocessing 数据预处理

#### Sklearn的datasets模块的数据集
- load_boston()、fetch_california_housing()：回归
- load_digits()、load_wine()：分类
- load_iris()、load_breast_cancer()：分类、聚类

In [1]:
from sklearn.datasets import load_breast_cancer

In [2]:
cancer = load_breast_cancer()

**加载后的数据集可视为一个字典，基本都有data（数据）、target（标签）、feature_names（特征名）、DESCR（描述信息）键**

In [3]:
len(cancer)

5

In [5]:
cancer_data = cancer['data']
cancer_data.shape

(569, 30)

In [7]:
cancer_name = cancer['feature_names']
cancer_name

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [9]:
cancer_target = cancer['target']
cancer_target.shape

(569,)

### 数据集划分
- 训练集 train：用于估计模型
- 验证集 validation：确定网络结构或控制模型复杂程度的参数（不懂）
- 测试集test：检验最优模型性能

数据量大时，典型划分方式各占 0.5,0.25,0.25

有一种**K折交叉验证法**
> from sklearn.model_selection import KFold

- 使用model_seelection模块的train_test_split()方法
> 有时会看到有cross_validation的使用，但**cross_validation模块在0.18版本中被弃用，现在已经被model_selection代替**。所以在导入的时候把"sklearn.cross_validation import  train_test_split "更改为  "from sklearn.model_selection import  train_test_split"

#### train_test_split函数参数
- *arrays：接收一个或多个待划分的数据集
    - 分类、回归，则分别传入data、target数据集
    - 聚类，则传入data数据集
    
- test_size：代表测试集大小
    - 传入float（0~1）表示占比
    - 传入int表示测试集记录的绝对数目
    - test_size、train_size都默认25%
- train_size：训练集大小，同上
- random_state：接收int，随机种子编号，不同随机种子编号产生不同随机结果
- shuffle：接收bool型，是否有放回抽样，True表示有放回抽样，此时stratify不能为空
- stratify：接收array/None。若为array，使用传入的标签进行分层抽样

In [15]:
from sklearn.model_selection import train_test_split
cancer_data_train,cancer_data_test,cancer_target_train,cancer_target_test = train_test_split(cancer_data,cancer_target,test_size=0.2,random_state=40)

In [16]:
cancer_data_test.shape

(114, 30)

In [17]:
cancer_data_train.shape

(455, 30)

### 使用转换器进行数据预处理、降维
数据预处理模块**preprocessing**
其中部分处理函数（标准化、归一化、二值化）：
- MinMaxScaler()：离差标准化
- StandardScaler()：特征标准差标准化
- Normalizer()：特征归一化
- Binarizer()：定量特征二值化
- OneHotEncoder()：定性特征独热编码处理
- FunctionTransformer()：特征自定义函数变换

> 降维是什么鬼？
> 处理一组高维数据(n*m维向量)，当n、m都非常大时，处理非常难，为了更快地处理数据采取降低数据的维度m

PCA降维（主成分分析）：提取数据的主要特征分量

`from sklearn.decomposition import PCA`
常用参数：
- n_components：None，所有特征都会保留；int，原始数据会降维到n个维度；float，会根据样本特征方差来决定降维后维度数；"mle"，会根据MLE算法依特征的方差分布情况自动选择一定数量的主成分特征来降维
- copy=False：运行算法时，是否将原数据复制一份。True，运行后原始数据不会变
- whiten=False：是否白化，即对降维后的数据的每个特征进行归一化，让方差都为1
- svd_solver="auto"：代表使用的SVD算法
    - full
    - arpack
    - randomized
    - auto：自动权衡选择以上三种

In [18]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

##通过训练集生成离差标准化规则
scaler = MinMaxScaler().fit(cancer_data_train)
##将规则分别运用于训练集、测试集
cancer_train_scaler = scaler.transform(cancer_data_train)
cancer_test_scaler = scaler.transform(cancer_data_test)
# cancer_test_scaler.mean()

0.2512257517694716

In [31]:
##PCA降维规则
pca_model = PCA(n_components=15).fit(cancer_train_scaler)
##规则分别应用于训练集、测试集
cancer_train_pca = pca_model.transform(cancer_train_scaler)
cancer_test_pca = pca_model.transform(cancer_test_scaler)
print('降维前 ',cancer_train_scaler.shape)
cancer_train_pca.shape

降维前  (455, 30)


(455, 15)

In [28]:
cancer_train_pca[0]

array([ 0.35554047,  0.42156407, -0.04121754, -0.37096383, -0.17949375,
        0.01576688, -0.0892475 ,  0.00893679,  0.07159521,  0.08121706,
        0.0090992 , -0.06418775,  0.05750302, -0.04087269, -0.01249581])